<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [33]:
# normalize = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]
normalize = [(0.12, 0.12, 0.12), (0.19, 0.19, 0.19)]

def get_oct_test_simclr_pipeline_transform():
    """Return a set of data augmentation transformations as described in the SimCLR paper."""
    data_transforms = transforms.Compose(
        [
            # transforms.Resize(size=(224, 224)),
            transforms.Resize(size=(256, 256)),
            transforms.RandomResizedCrop(size=224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(*normalize),
        ]
    )
    return data_transforms


def get_oct_simclr_pipeline_transform():
    """Return a set of data augmentation transformations as described in the SimCLR paper."""
    color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)
    data_transforms = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.RandomResizedCrop(size=224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([color_jitter], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize(*normalize),
        ]
    )
    return data_transforms


def get_oct_data_loaders(root_path, batch_size=32):
    train_dataset = datasets.ImageFolder(f"{root_path}/train", transform=get_oct_simclr_pipeline_transform())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8, drop_last=False, shuffle=True)

    test_dataset = datasets.ImageFolder(f"{root_path}/test", transform=get_oct_test_simclr_pipeline_transform())

    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=8, drop_last=False, shuffle=True)

    val_dataset = datasets.ImageFolder(f"{root_path}/val", transform=get_oct_test_simclr_pipeline_transform())

    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=8, drop_last=False, shuffle=True)
    return train_loader, test_loader, val_loader

In [34]:
with open("./config.yml") as file:
    config = yaml.load(file, Loader=yaml.UnsafeLoader)

In [35]:
if config.arch == "resnet18":
    model = torchvision.models.resnet18(pretrained=False, num_classes=4).to(device)
elif config.arch == "resnet50":
    model = torchvision.models.resnet50(pretrained=False, num_classes=4).to(device)

In [36]:
checkpoint = torch.load(config.checkpoint_path, map_location=device)
state_dict = checkpoint["state_dict"]

for k in list(state_dict.keys()):
    if k.startswith("backbone."):
        if k.startswith("backbone") and not k.startswith("backbone.fc"):
            # remove prefix
            state_dict[k[len("backbone.") :]] = state_dict[k]
    del state_dict[k]

In [37]:
log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [38]:
if config.dataset_name == "oct":
    train_loader, test_loader, val_loader = get_oct_data_loaders(config.dataset_path, config.batch_size)
print("Dataset:", config.dataset_name)

Dataset: oct


In [39]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [40]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [41]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [42]:
train_loss_arr, train_acc_arr, test_acc_arr, val_acc_arr = [], [], [], []
from tqdm import tqdm
import csv

epochs = 100
for epoch in range(epochs):
    top1_train_accuracy = 0
    train_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        top1 = accuracy(logits, y_batch, topk=(1,))
        top1_train_accuracy += top1[0].item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss_arr.append(train_loss / len(train_loader))
    train_acc_arr.append(top1_train_accuracy / len(train_loader))


    top1_accuracy = 0
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        logits = model(x_batch)
        top1 = accuracy(logits, y_batch, topk=(1,))
        top1_accuracy += top1[0].item()
    test_acc_arr.append(top1_accuracy / len(test_loader))

    top1_accuracy = 0
    for x_batch, y_batch in val_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        logits = model(x_batch)
        top1 = accuracy(logits, y_batch, topk=(1,))
        top1_accuracy += top1[0].item()
    val_acc_arr.append(top1_accuracy / len(val_loader))

    print(f"Epoch: {epoch}, train_loss: {train_loss_arr[-1]}, train_acc: {train_acc_arr[-1]}, test_acc: {test_acc_arr[-1]}, val_acc: {val_acc_arr[-1]}")

root_path = "./csv"
name = config.name
if not os.path.exists(root_path):
    os.makedirs(root_path)
    
with open(f"{root_path}/{name}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["train_loss", "train_acc", "test_acc", "val_acc"])
    for i in range(len(train_loss_arr)):
        writer.writerow([train_loss_arr[i], train_acc_arr[i], test_acc_arr[i], val_acc_arr[i]])

Epoch: 0, train_loss: 1.4383950630823772, train_acc: 26.767676035563152, test_acc: 34.693878173828125, val_acc: 45.238094329833984
Epoch: 1, train_loss: 1.3258229494094849, train_acc: 39.47811381022135, test_acc: 42.85714340209961, val_acc: 42.85714340209961
Epoch: 2, train_loss: 1.229421893755595, train_acc: 43.07922871907552, test_acc: 44.89795684814453, val_acc: 42.85714340209961
Epoch: 3, train_loss: 1.1917271216710408, train_acc: 48.4217160542806, test_acc: 38.775508880615234, val_acc: 42.85714340209961
Epoch: 4, train_loss: 1.147494951883952, train_acc: 49.60279846191406, test_acc: 44.89795684814453, val_acc: 40.47618865966797
Epoch: 5, train_loss: 1.1102863152821858, train_acc: 53.70633316040039, test_acc: 53.06122589111328, val_acc: 38.095237731933594
Epoch: 6, train_loss: 1.0957211256027222, train_acc: 52.372684478759766, test_acc: 40.81632614135742, val_acc: 47.619049072265625
Epoch: 7, train_loss: 1.0877915223439534, train_acc: 52.13068135579427, test_acc: 34.693878173828125